In [1]:
import numpy as np
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

In [2]:
structure = input("Enter the number of neurons of each layer in your network").split(" ")
structure = [int(x) for x in structure]
print(structure)

[4, 2, 3]


In [3]:
data = load_iris()["data"]
target = load_iris()["target"]

In [4]:
data = data[[2, 57, 105]]
target = target[[2, 57, 105]]

In [5]:
for x, y in zip(data, target):
    print(x, y)

[4.7 3.2 1.3 0.2] 0
[4.9 2.4 3.3 1. ] 1
[7.6 3.  6.6 2.1] 2


In [6]:
one_hot_labels = np.zeros((len(data), 3))
for i in range(len(one_hot_labels)):
    for j in range(len(one_hot_labels[i])):
        one_hot_labels[i, target[i]] = 1
print(one_hot_labels)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [7]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

def softmax(x):
    exponential_sum = np.sum([np.exp(i) for i in x])
    exponential_x = np.array([np.exp(i) / exponential_sum for i in x])
    return exponential_x

def calculate_loss(actual, expected):
    loss = - np.sum(actual.reshape(-1) * expected)
    return loss

In [8]:
actual = np.array([[0.08], [0.24], [0.72]])
expected = np.array([0, 0, 1])
calculate_loss(actual, expected)

-0.72

In [60]:
H = len(structure) - 2
learning_rate = 0.10
epochs = 3
all_epoch_errors = []

for epoch in range(epochs):
    each_epoch_errors = []
    
    np.random.seed(2023)
    w_n = [np.random.randn(l, next_l) for l,next_l in zip(structure[:-1], structure[1:])] # (4,2), (2,3)
    b_n = [np.random.randn(l, 1) for l in structure[1:]]  # (2,1), (3,1)
    
    for x, y in zip(data, one_hot_labels):
        z_n, a_n = [], []
        
        for i, (b, w) in enumerate(zip(b_n, w_n)):
            if i == 0:
                z = np.dot(w.T, x).reshape(-1,1) + np.array(b)
                a = sigmoid(z)                
                
            elif (i > 0) and (i < H):
                z = np.dot(w.T, a)
                a = sigmoid(z)
            else:
                z = np.dot(w.T, a)
                a = softmax(z)
            
            """    
            print(z)
            print(a)
            print()
            """                
            z_n.append(z)
            a_n.append(a)

        # BackPropagate
        np.random.seed(2023)
        e_je_w_ns = [np.zeros(w.shape) for w in w_n]
        e_je_b_ns = [np.zeros(b.shape) for b in b_n]
                
        datapoint_errors = []
        
        for L in range(H, -1, -1):
            if L != H:
                delta = sigmoid_derivative(z_n[L]) * np.dot(w_n[L+1], delta)
            else:
                # print(y.reshape(-1,1).shape)
                delta = a_n[L] - y.reshape(-1, 1)   # (3,1)                
            e_je_b_ns = delta
            
            if L != 0:
                e_je_w_ns[L] = np.dot(a_n[L], delta.T)
                print(f"delta.T shape : {delta.T.shape}")
                print(f"a_n[L] shape : {a_n[L].shape}")                
            else:
                print(f"x : {x.reshape(-1,1).shape}")
                print(f"delta : {delta.T.shape}")
                e_je_w_ns[L] = np.dot(x.reshape(-1,1), delta.T)   # (4,1), (1,2), (4,2)          
                
            print(f"{epoch}th epoch e_je_w_ns :\n {e_je_w_ns}")
            print(f"{epoch}th epoch e_je_b_ns :\n {e_je_b_ns}")
            print()

delta.T shape : (1, 3)
a_n[L] shape : (3, 1)
0th epoch e_je_w_ns :
 [array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]]), array([[-0.24950084,  0.07615482,  0.17334602],
       [-0.06964012,  0.02125617,  0.04838396],
       [-0.15851706,  0.04838396,  0.1101331 ]])]
0th epoch e_je_b_ns :
 [[-0.47765802]
 [ 0.14579494]
 [ 0.33186308]]

x : (4, 1)
delta : (1, 2)
0th epoch e_je_w_ns :
 [array([[-0.95120048, -0.04562449],
       [-0.64762586, -0.03106348],
       [-0.26309801, -0.01261954],
       [-0.04047662, -0.00194147]]), array([[-0.24950084,  0.07615482,  0.17334602],
       [-0.06964012,  0.02125617,  0.04838396],
       [-0.15851706,  0.04838396,  0.1101331 ]])]
0th epoch e_je_b_ns :
 [[-0.20238308]
 [-0.00970734]]

delta.T shape : (1, 3)
a_n[L] shape : (3, 1)
0th epoch e_je_w_ns :
 [array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]]), array([[ 0.51576751, -0.69143711,  0.17566959],
       [ 0.02673245, -0.03583748,  0.00910503],
       [ 0.175669

In [27]:
z_n

[array([[ 6.27374581],
        [-6.78636791]]),
 array([[ 0.09834887],
        [-3.11682648],
        [-1.07497685]])]

In [28]:
a_n

[array([[0.99811839],
        [0.00112779]]),
 array([[0.74102356],
        [0.02975083],
        [0.22922561]])]

In [29]:
a0, a1, a2 = np.exp(0.09834887), np.exp(-3.11682648), np.exp(-1.07497685)
a = a0 + a1 + a2
print(a0, a1, a2)
print(a0/a, a1/a, a2/a)

1.1033476428641078 0.04429752467070435 0.34130565643548416
0.7410235617601022 0.029750831227981552 0.22922560701191635


In [30]:
softmax(z_n[1])

array([[0.74102356],
       [0.02975083],
       [0.22922561]])